In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint

import requests
import time
# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"


In [2]:
# Import wine database
# Data source: https://www.kaggle.com/zynicide/wine-reviews#winemag-data_first150k.csv
wine_data = pd.read_csv("./Resources/winemag-data-130k-v2.csv")

In [3]:
# Raw data for wine database
wine_data.head()
wine_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
Unnamed: 0               129971 non-null int64
country                  129908 non-null object
description              129971 non-null object
designation              92506 non-null object
points                   129971 non-null int64
price                    120975 non-null float64
province                 129908 non-null object
region_1                 108724 non-null object
region_2                 50511 non-null object
taster_name              103727 non-null object
taster_twitter_handle    98758 non-null object
title                    129971 non-null object
variety                  129970 non-null object
winery                   129971 non-null object
dtypes: float64(1), int64(2), object(11)
memory usage: 13.9+ MB


In [4]:
# Clean data, select US wine data
US_wine_data=wine_data[["country", "province", "winery", "points","price", "variety", "title"]].set_index("country").loc["US"]
US_wine_data=US_wine_data.rename(columns={"province":"State_full"})

US_wine_data["State_full"]=US_wine_data["State_full"].replace({"America":np.NaN})
US_wine_data=US_wine_data.sort_values("State_full")
US_wine_data=US_wine_data.dropna(subset=["State_full"])

# US_wine_data["year"]= US_wine_data["title"].str.extract('(\d{4})')
# US_wine_data=US_wine_data.dropna(subset=["year"])
# US_wine_data=US_wine_data.sort_values("year")
US_wine_data


,State_full,winery,points,price,variety,title
country,,,,,,
US,Arizona,Fiddlebender,84,13.0,White Blend,Fiddlebender NV Out of Sight White (Arizona)
US,Arizona,Page Springs,84,30.0,Viognier,Page Springs 2007 La Serrana Viognier (Cochise...
US,Arizona,Page Springs,85,21.0,Malvasia Bianca,Page Springs 2011 Vino de la Familia Blanca Ma...
US,Arizona,Pillsbury,84,38.0,Symphony,Pillsbury 2011 Symphony (Cochise County)
US,Arizona,Page Cellars,85,47.0,Red Blend,Page Cellars 2010 Landscape Page Springs Estat...
US,Arizona,Page Springs,87,16.0,Red Blend,Page Springs 2011 Vino del Barrio Red (Cochise...
US,Arizona,Fiddlebender,85,16.0,Red Blend,Fiddlebender NV In Tune Red (Arizona)
US,Arizona,Sultry Cellars,86,54.0,Red Blend,Sultry Cellars 2010 Inked Red (Arizona)
US,Arizona,Dos Cabezas,82,20.0,Red Blend,Dos Cabezas 2005 El Norte Red (Cochise County)


In [5]:
# US_wine_data_winery=US_wine_data.set_index("winery")
US_wine_data_winery=US_wine_data.groupby(["winery","State_full"])["price","points"].mean()
US_wine_data_winery=US_wine_data_winery.rename(columns={"price":"Average_price",
                                                       "points":"Average_points"})

US_wine_data_winery=US_wine_data_winery.reset_index()
US_wine_data_winery["Average_price"]=US_wine_data_winery["Average_price"].map("${:2f}".format)
# US_wine_data_winery["mean_points"]=US_wine_data_winery.groupby(["winery"])["points"].mean()
US_wine_data_winery.head(20)
# US_wine_data_winery.describe()

,winery,State_full,Average_price,Average_points
0,10 Knots,California,$24.750000,83.250000
1,100 Percent Wine,California,$18.000000,86.333333
2,1000 Stories,California,$19.000000,90.500000
3,1070 Green,California,$25.000000,88.000000
4,10Span,California,$12.000000,83.500000
5,12C Wines,California,$92.600000,92.600000
6,14 Hands,Washington,$17.412698,87.507463
7,16X20,California,$40.000000,85.500000
8,1789 Wines,Oregon,$48.000000,91.000000
9,181,California,$15.000000,84.000000


In [6]:
# maybe bin the data? Max 98, Min 80
# 80, 85, 90, 95, 100
US_wine_data_winery["Average_points"].min()

80.0

In [7]:
# US_wine_data.info()
US_wine_data_winery.describe()

,Average_points
count,5495.000000
mean,87.829100
std,2.767866
min,80.000000
25%,86.000000
50%,87.857143
75%,90.000000
max,98.000000


In [8]:
# Database showing Latitude and Longitidue for each State
# Data source: https://www.kaggle.com/washimahmed/usa-latlong-for-state-abbreviations#statelatlong.csv
state_data = pd.read_csv("./Resources/statelatlong.csv")
state_data.head()

,State,Latitude,Longitude,City
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California


In [9]:
state_data=state_data.rename(columns={"State":"State_abbr","City":"State_full",
                                     "Latitude":"State_lat", "Longitude":"State_lng"})
state_data

,State_abbr,State_lat,State_lng,State_full
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California
5,CO,38.997934,-105.550567,Colorado
6,CT,41.518784,-72.757507,Connecticut
7,DE,39.145251,-75.418921,Delaware
8,DC,38.899349,-77.014567,District of Columbia
9,FL,27.975728,-83.833017,Florida


In [10]:
# JSON Database acquiring (current) weather data for each state
# Data source: https://openweathermap.org
lat_lngs = []
cities = []
lat_lngs = zip(state_data["State_lat"], state_data["State_lng"])
# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)


In [11]:
units = "imperial"
url = "http://api.openweathermap.org/data/2.5/weather?"+ "appid=" + api_key + "&units=" + units
cityinfo=[]

record_count=0
set_count=1
print("Beginning Data Retrieval")
print("-----------------------------")

for city in cities:
    try:
        query_url = url + "&q=" + city 
        weather_response = requests.get(query_url)
        weather_json = weather_response.json()
#         citynames.append(weather_json['name'])        
        record_count=record_count+1
        if record_count==51:
            set_count=set_count+1
            record_count=0
            time.sleep(1.001)
        print(f'Processing Record {record_count} of Set {set_count} | {weather_json["name"]}')
        cityinfo.append(weather_json)   
    except (KeyError, IndexError):
        record_count=record_count-1
        print("City not found. Skipping...")
print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")

Beginning Data Retrieval
-----------------------------
Processing Record 1 of Set 1 | Prattville
Processing Record 2 of Set 1 | Bethel
Processing Record 3 of Set 1 | Camp Verde
Processing Record 4 of Set 1 | Sherwood
Processing Record 5 of Set 1 | Clovis
Processing Record 6 of Set 1 | Canon City
Processing Record 7 of Set 1 | Meriden
Processing Record 8 of Set 1 | Dover
Processing Record 9 of Set 1 | Washington DC.
Processing Record 10 of Set 1 | Clearwater
Processing Record 11 of Set 1 | Dublin
Processing Record 12 of Set 1 | Honolulu
Processing Record 13 of Set 1 | Hamilton
Processing Record 14 of Set 1 | Springfield
Processing Record 15 of Set 1 | Plainfield
Processing Record 16 of Set 1 | Ames
Processing Record 17 of Set 1 | Great Bend
Processing Record 18 of Set 1 | Elizabethtown
Processing Record 19 of Set 1 | Zachary
Processing Record 20 of Set 1 | Hermon
Processing Record 21 of Set 1 | Burke
Processing Record 22 of Set 1 | Uxbridge
Processing Record 23 of Set 1 | Traverse City


In [12]:
statedata_pd=pd.DataFrame()

for x in range(len(cityinfo)):
    statedata_pd.loc[x, 'City'] = cityinfo[x]['name']
    statedata_pd.loc[x, 'State'] = state_data['State_full'][x]
    statedata_pd.loc[x, 'Cloudiness'] = cityinfo[x]['clouds']['all']
    statedata_pd.loc[x, 'Country'] = cityinfo[x]['sys']['country']
    statedata_pd.loc[x, 'Date'] = cityinfo[x]['dt']
    statedata_pd.loc[x, 'Humidity'] = cityinfo[x]['main']['humidity']
    statedata_pd.loc[x, 'Lat'] = cityinfo[x]['coord']['lat']
    statedata_pd.loc[x, 'Lng'] = cityinfo[x]['coord']['lon']
    statedata_pd.loc[x, 'Max Temp'] = cityinfo[x]['main']['temp_max']
    statedata_pd.loc[x, 'Wind Speed'] = cityinfo[x]['wind']['speed']

statedata_pd['Cloudiness'] = statedata_pd['Cloudiness'].astype(int)
statedata_pd['Date'] = statedata_pd['Date'].astype(int)
statedata_pd['Humidity'] = statedata_pd['Humidity'].astype(int)

statedata_pd.count()
statedata_pd.head()

,City,State,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Prattville,Alabama,1,US,1550799300,83,32.46,-86.46,75.2,3.36
1,Bethel,Alaska,1,US,1550796960,79,60.79,-161.76,30.2,11.41
2,Camp Verde,Arizona,90,US,1550800140,100,34.56,-111.85,32.0,6.93
3,Sherwood,Arkansas,90,US,1550799480,86,34.82,-92.22,42.8,10.29
4,Clovis,California,1,US,1550799300,57,36.83,-119.70,53.6,2.59


In [13]:
# Clean and denormalize State weather data
statedata_pd.reset_index(drop=False)
statedata_weather=statedata_pd[["State", "Max Temp", "Humidity", "Wind Speed"]]
statedata_weather=statedata_weather.rename(columns={"State":"State_full"})
statedata_weather.head()

,State_full,Max Temp,Humidity,Wind Speed
0,Alabama,75.2,83,3.36
1,Alaska,30.2,79,11.41
2,Arizona,32.0,100,6.93
3,Arkansas,42.8,86,10.29
4,California,53.6,57,2.59


In [14]:
# Merge State info and State weather data
merge_state_weather_table = pd.merge(state_data, statedata_weather, on="State_full")
merge_state_weather_table.head()

,State_abbr,State_lat,State_lng,State_full,Max Temp,Humidity,Wind Speed
0,AL,32.601011,-86.680736,Alabama,75.2,83,3.36
1,AK,61.302501,-158.775020,Alaska,30.2,79,11.41
2,AZ,34.168219,-111.930907,Arizona,32.0,100,6.93
3,AR,34.751928,-92.131378,Arkansas,42.8,86,10.29
4,CA,37.271875,-119.270415,California,53.6,57,2.59


In [15]:
complete_merge_table = pd.merge(US_wine_data_winery, merge_state_weather_table, on= "State_full")
complete_merge_table = complete_merge_table.drop(["State_abbr","State_lat","State_lng"], axis=1)
complete_merge_table

,winery,State_full,Average_price,Average_points,Max Temp,Humidity,Wind Speed
0,10 Knots,California,$24.750000,83.250000,53.60,57,2.59
1,100 Percent Wine,California,$18.000000,86.333333,53.60,57,2.59
2,1000 Stories,California,$19.000000,90.500000,53.60,57,2.59
3,1070 Green,California,$25.000000,88.000000,53.60,57,2.59
4,10Span,California,$12.000000,83.500000,53.60,57,2.59
5,12C Wines,California,$92.600000,92.600000,53.60,57,2.59
6,16X20,California,$40.000000,85.500000,53.60,57,2.59
7,181,California,$15.000000,84.000000,53.60,57,2.59
8,1850,California,$24.000000,89.333333,53.60,57,2.59
9,24 Knots,California,$17.000000,82.000000,53.60,57,2.59


In [16]:
complete_merge_table.describe()

,Average_points,Max Temp,Humidity,Wind Speed
count,5491.000000,5491.000000,5491.000000,5491.000000
mean,87.828566,52.418376,58.714442,3.666685
std,2.768711,4.224885,5.435400,2.379653
min,80.000000,-4.010000,31.000000,2.260000
25%,86.000000,53.600000,57.000000,2.590000
50%,87.846154,53.600000,57.000000,2.590000
75%,90.000000,53.600000,57.000000,2.590000
max,98.000000,78.800000,100.000000,14.990000


In [17]:
complete_merge_table.to_excel("wine_weather_analysis_output.xlsx")